In [1]:
import pandas as pd
import numpy as np
import json  
import urllib.request as urllib2
import requests
from dicttoxml import dicttoxml
from lxml import etree
from IPython.display import display

In [2]:
client_access_token = '' # GET A DEVELOPER KEY FROM GENIUS API

In [ ]:
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="

for pcount in range(16,5000): 
    with open('Songs/songs'+str(pcount)+'.txt', 'r') as infile:
        outfile = open('Genius/genius'+str(pcount)+'.txt', 'w')
        print('No success for songs:')
        for line in infile:
            song = infile.readline()

            try:
                # Separating artist information
                song = song.split(';')
                artist_name = song[0]
                search_term = song[1]
                
                # Requesting song search
                querystring = _URL_API + _URL_SEARCH + urllib2.quote(search_term)
                request = urllib2.Request(querystring)
                request.add_header("Authorization", "Bearer " + client_access_token)
                request.add_header("User-Agent", "") 
                
                # Getting song id of first song from response
                response = urllib2.urlopen(request, timeout=3)
                raw = response.read()
                json_obj = json.loads(raw)
                song_id = None
                for i in range(len(json_obj['response']['hits'])):
                    if json_obj['response']['hits'][i]['result']['primary_artist']['name'] == artist_name:
                        song_id = json_obj['response']['hits'][i]['result']['id']
                        break
                
                # Getting song page
                querystring = "https://api.genius.com/songs/" + str(song_id)
                request = urllib2.Request(querystring)
                request.add_header("Authorization", "Bearer " + client_access_token)
                request.add_header("User-Agent", "")
                response = urllib2.urlopen(request, timeout=3)
                raw = response.read()
                json_obj = json.loads(raw)['response']['song']
                about = json_obj['description']['dom']
                
                # Getting song description from page
                desc = ''
                xml = dicttoxml(about, custom_root='children')
                q = etree.fromstring(xml)
                for child in q.findall(".//item[@type='str']"):
                    if child.text!=None:
                        desc += child.text
        
                outfile.write(desc+'\n\n')
            except:
                print(search_term[:-1]+', ', end='')
                
        outfile.close()
    print("\nDone Playlist "+str(pcount)+'\n')